Random Forest de Regressão é um modelo de aprendizado de máquina que utiliza uma combinação de múltiplas árvores de decisão para fazer previsões contínuas, como estimar valores numéricos. Funciona criando várias árvores de decisão a partir de diferentes subconjuntos de dados e variáveis, e a previsão final é obtida fazendo a média das previsões de todas essas árvores. Essa abordagem reduz o risco de overfitting e aumenta a precisão, pois a diversidade de árvores ajuda a capturar padrões complexos nos dados. O Random Forest é robusto, eficaz em lidar com grandes volumes de dados e menos sensível a outliers.

In [ ]:
import shap
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

### Aqui são criadas a features target e tambem selecionadas as features importantes para o a previsão do modelo

In [ ]:
df_novo = pd.read_csv('player.csv')

# Cria a coluna de gols binário para selecionar quem fex mais de 0 gols
df_novo['gols_binario'] = (df_novo['goals_overall'] > 0).astype(int)

#Cria a coluna de chance de fazer o primeiro gol que será o target
df_novo['chance_primeiro_gol'] = (df_novo['gols_binario'] * df_novo['minutes_played_overall'] * df_novo['shots_per90_percentile_overall']/10000)

# Reune as features que serão usadas para treinar o modelo
important_features = [
    'minutes_played_overall', 'shots_per90_percentile_overall',
    'goals_per90_percentile_overall', 'assists_overall',
    'annual_salary_eur_percentile', 'minutes_played_home',
    'booked_over05_overall', 'minutes_played_away',
    'clean_sheets_percentage_percentile_overall',
    'cards_per90_percentile_overall',
    'penalty_goals', 'shirt_number', 
    'three_goals_in_a_game_total_overall',
    'booked_over05_percentage_overall', 'goals_home',
    'goals_away', 'min_per_card_percentile_overall',
    'sm_goals_conceded_total_overall', 'sm_goals_scored_total_overall',
    'two_goals_in_a_game_total_overall'
]

# Separa as features (X) e o target (y)
X = df_novo[important_features]
y = df_novo['chance_primeiro_gol']


### Iniciação da limpeza dos dados X e y

In [ ]:
numericas = X.select_dtypes(include=['float64', 'int64']).columns
categoricas = X.select_dtypes(include=['object']).columns
imputer_numerico = SimpleImputer(strategy='median')
imputer_categorico = SimpleImputer(strategy='most_frequent')


X[numericas] = imputer_numerico.fit_transform(X[numericas])

# Preenche valores nulos nas colunas categóricas, se houver
if not categoricas.empty:
    X[categoricas] = imputer_categorico.fit_transform(X[categoricas])

# Normaliza as colunas numéricas entre 0 e 1 usando MinMaxScaler
scaler = MinMaxScaler()
X[numericas] = scaler.fit_transform(X[numericas])

X.sample(10)

In [ ]:
# Trata os valores nulos em y com simpleImputer
imputer_numerico = SimpleImputer(strategy='median')

# Convertendo y para numpy e preenchendo valores nulos
y = y.to_numpy().reshape(-1, 1)
y_imputado = imputer_numerico.fit_transform(y)


### Inicia a escolha dos times que serão previstos

In [ ]:
team_a = "São Paulo"
team_b = "Botafogo"

df_teams = df_novo[(df_novo['Current Club'] == team_a) | (df_novo['Current Club'] == team_b)].copy()

# Verifica se temos jogadores dos dois times pela quantidade
print(f"Jogadores do {team_a} e {team_b}: {df_teams.shape[0]}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_imputado.flatten(), test_size=0.3, random_state=42)


### Adiciono o GridSearchCV para verificar os melhores hiperparâmetros do modelo

In [ ]:
# Definir os hiperparâmetros a serem ajustados
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40],
}

model = RandomForestRegressor(random_state=42)

# Configurar o GridSearch
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2)

# Ajusta o GridSearch ao conjunto de treinamento
grid_search.fit(X_train, y_train)

# Exibe os melhores hiperparâmetros
print("Melhores parâmetros encontrados: ", grid_search.best_params_)

best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

## Ajustes de hiperparametros para iniciação do modelo preditivo

In [ ]:
# Adicionando os hiperparâmetros ajustados ao modelo
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

# Treinando o modelo
model.fit(X_train, y_train)


predicoes_teams = best_model.predict(X_test)


predicoes_df = pd.DataFrame(predicoes_teams, columns=['Previsão de Gols'], index=X_test.index)

res = X_test.join(predicoes_df)

resultado_teams_filtrado = res[res['Previsão de Gols'] >= 1]


resultado_teams_ordenado = resultado_teams_filtrado.sort_values(by='Previsão de Gols', ascending=False)

In [ ]:
predicoes_df = pd.DataFrame(predicoes_teams, columns=['Previsão de Gols'], index=X_test.index)

res = X_test.join(predicoes_df)

# Filtra jogadores com previsão de gols >= 1
resultado_teams_filtrado = res[res['Previsão de Gols'] >= 1]

# Ordena os jogadores pela previsão de gols
resultado_teams_ordenado = resultado_teams_filtrado.sort_values(by='Previsão de Gols', ascending=False)

resultado_teams_ordenado

In [ ]:
predicoes_teams = model.predict(X_test)
len(predicoes_teams)
predicoes_teams

In [ ]:
# Defini a função sigmoide para que os resultados fiquem entre 0 e 1
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

probabilities = sigmoid(resultado_teams_ordenado['Previsão de Gols'])

resultado_teams_ordenado['Probabilidade'] = probabilities

df_probabilidades = resultado_teams_ordenado[['Probabilidade']].copy()

df_probabilidades = df_probabilidades.merge(df_novo[['full_name', 'Current Club']], 
                                            left_index=True, right_index=True, how='left')



In [ ]:
df_probabilidades = df_probabilidades[(df_probabilidades['Current Club'] == team_a) | (df_probabilidades['Current Club'] == team_b)]
top = df_probabilidades.sort_values(by='Probabilidade', ascending=False).head(20)
top


Verificação das metricas do modelo

In [ ]:
# Calcula as métricas de desempenho
mse = mean_squared_error(y_test, predicoes_teams)
r2 = r2_score(y_test, predicoes_teams)
mae = mean_absolute_error(y_test, predicoes_teams)


print(f'Erro Absoluto Médio (MAE): {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

In [ ]:
# Criar o explicador SHAP para o modelo
explainer = shap.TreeExplainer(model)

# Calcular os valores SHAP para o conjunto de dados filtrado
shap_values = explainer.shap_values(X_train)

# Visualizar um resumo gráfico dos valores SHAP para entender as principais variáveis
shap.summary_plot(shap_values, X_train, feature_names=X_train.columns)